# Common table expressions

Using the northwind database

In [23]:
# I downloaded the database from https://en.wikiversity.org/wiki/Database_Examples/Northwind/MySQL
# saved it as northwind.sql
# 
# Using SQLAlchemy
from sqlalchemy import create_engine

# Database connection settings
host = "localhost"  # host
user = "root"  #  MySQL username
password = "Malcomx1"  #  MySQL password
database = "northwind"  #  database name


# creates a SQLAlchemy engine, which serves as the entry point for interacting with the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

print(f"Connected to the {database} database successfully!")

Connected to the northwind database successfully!


## 1. Write a CTE that lists the names and quantities of products with a unit price greater than $50.

In [32]:
# there is no 'quantities' in the database. 

import pandas as pd
from sqlalchemy import create_engine, text

query = text("""
WITH ProductGreaterThan50 AS (
    SELECT 
        ProductName,
        -- quantity_per_unit,
        Unit,
        Price
    FROM 
        products
    WHERE 
        Price > 50
)
SELECT 
    ProductName,
    -- quantity_per_unit,
    Unit,
    Price
    
FROM 
    ProductGreaterThan50;
""")

# Use a connection from the engine
with engine.connect() as connection:
    df_question1 = pd.read_sql(query, connection)

# Display the results
print(df_question1)


               ProductName                  Unit  Price
0          Mishi Kobe Niku      18 - 500 g pkgs.   97.0
1         Carnarvon Tigers            16 kg pkg.   63.0
2   Sir Rodney's Marmalade         30 gift boxes   81.0
3  Thüringer Rostbratwurst  50 bags x 30 sausgs.  124.0
4            Côte de Blaye    12 - 75 cl bottles  264.0
5    Manjimup Dried Apples      50 - 300 g pkgs.   53.0
6     Raclette Courdavault             5 kg pkg.   55.0


#### Expected results

|ProductName                     |Unit|
|--------------------------------|----|
|Mishi Kobe Niku                 |18 - 500 g pkgs.|
|Carnarvon Tigers                |16 kg pkg.|
|Sir Rodney's Marmalade          |30 gift boxes|
|Thüringer Rostbratwurst         |50 bags x 30 sausgs.|
|Côte de Blaye                   |12 - 75 cl bottles|
|Manjimup Dried Apples           |50 - 300 g pkgs.|
|Raclette Courdavault            |5 kg pkg.|


## 2. What are the top 5 most profitable products?

In [ ]:
# 'Profit' is defined incorrectly, but whatever. 
# TotalRevenue defined as quantity * price

import pandas as pd
from sqlalchemy import create_engine, text

# query = text("""
# SELECT 
#     p.ProductName AS ProductName,
#     p.id AS ProductID,
#     (od.Quantity * p.Price) AS TotalRevenue
# FROM 
#     products p
# INNER JOIN 
#     orderdetails od
# ON 
#     p.id = od.product_id
# ORDER BY 
#     TotalRevenue DESC
# LIMIT 5;
# """)


# query = text("""
# SELECT 
#     ProductName, 
#     Price 
#     -- standard_cost, 
#     -- (list_price - standard_cost) AS TotalRevenue
# FROM 
#     products
# ORDER BY 
#     -- profit DESC
#     Price DESC
# LIMIT 5;
# """)

# query = text("""
# SELECT
#     p.ProductID,
#     p.ProductName,
#     od.Quantity
# FROM
#     products AS p
# JOIN
#     orderdetails AS od
# ON
#     p.ProductID = od.ProductID;
# """)

query = text("""
SELECT
    products.ProductID,
    products.ProductName,
    SUM(orderdetails.Quantity * products.Price) AS TotalRevenue
FROM
    products
JOIN
    orderdetails
ON
    products.ProductID = orderdetails.ProductID
GROUP BY
    products.ProductID, products.ProductName
ORDER BY
    TotalRevenue DESC
LIMIT 5;
    
""")




# Use a connection from the engine
with engine.connect() as connection:
    df_question2 = pd.read_sql(query, connection)

# Display the results
print(df_question2)


   ProductID              ProductName  TotalRevenue
0         38            Côte de Blaye       63096.0
1         29  Thüringer Rostbratwurst       20832.0
2         59     Raclette Courdavault       19030.0
3         62           Tarte au sucre       15925.0
4         60        Camembert Pierrot       14620.0


#### Expected results
Alice Mutton is 6th

|ProductID                       |ProductName|TotalRevenue|
|--------------------------------|-----------|------------|
|-                             |-|-       |
|-                           |-|-       |
|-                            |-|-       |
|-                             |-|-       |
|-                              |-|-       |
|17                              |Alice Mutton|12909       |

## 3. Write a CTE that lists the top 5 categories by the number of products they have.

In [54]:
import pandas as pd
from sqlalchemy import create_engine, text


query = text("""
WITH CategoryProductCount AS (
    SELECT
        categories.CategoryID,
        categories.CategoryName,
        COUNT(products.ProductID) AS ProductCount
    FROM
        categories
    JOIN
        products
    ON
        categories.CategoryID = products.CategoryID
    GROUP BY
        categories.CategoryID, categories.CategoryName
)
SELECT
    CategoryName,
    ProductCount
FROM
    CategoryProductCount
ORDER BY
    ProductCount DESC
LIMIT 5;


""")


# Use a connection from the engine
with engine.connect() as connection:
    df_question3 = pd.read_sql(query, connection)

# Display the results
print(df_question3)


     CategoryName  ProductCount
0     Confections            13
1       Beverages            12
2      Condiments            12
3         Seafood            12
4  Dairy Products            10


```sql
-- Your code down below
```

#### Expected result
|CategoryName                    |ProductCount|
|--------------------------------|------------|
|Confections                     |13          |
|Beverages                       |12          |
|Condiments                      |12          |
|Seafood                         |12          |
|Dairy Products                  |10          |


## 4. Write a CTE that shows the average order quantity for each product category.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

query = text("""
WITH AveOrderQtyCategory AS (
    SELECT
        categories.CategoryID,
        categories.CategoryName,
        AVG(orderdetails.Quantity) AS AvgOrderQuantity
    FROM
        categories
    JOIN
        products
    ON
        categories.CategoryID = products.CategoryID
    JOIN
        orderdetails
    ON
        products.ProductID = orderdetails.ProductID
    GROUP BY
        categories.CategoryID, categories.CategoryName
)
SELECT
    CategoryName,
    AvgOrderQuantity
FROM
    AveOrderQtyCategory
ORDER BY
    -- AvgOrderQuantity DESC
    CategoryName ASC -- alphabetical order
-- LIMIT 5;

""")


# Use a connection from the engine
with engine.connect() as connection:
    df_question4 = pd.read_sql(query, connection)

# Display the results
print(df_question4)


     CategoryName  AvgOrderQuantity
0       Beverages           24.6129
1      Condiments           28.2245
2     Confections           25.1190
3  Dairy Products           26.0100
4  Grains/Cereals           21.7143
5    Meat/Poultry           25.7600
6         Produce           21.6667
7         Seafood           21.5672


```sql
-- Your code down below

```

|CategoryName                    |AvgOrderQuantity|
|--------------------------------|----------------|
|Beverages                       |24.6129         |
|Condiments                      |28.2245         |
|Confections                     |25.1190         |
|Dairy Products                  |26.0100         |
|Grains/Cereals                  |21.7143         |
|Meat/Poultry                    |25.7600         |
|Produce                         |21.6667         |
|Seafood                         |21.5672         |


# 5. Create a CTE to calculate the average order amount for each customer.

In [85]:
import pandas as pd
from sqlalchemy import create_engine, text

# # Total Order Amount
# query = text("""
# SELECT
#     customers.CustomerID,
#     customers.CustomerName,
#     COUNT(orderdetails.OrderID) AS OrderCount,  -- count of orders
#     SUM(products.Price * orderdetails.Quantity) AS TotalOrderAmount
# FROM
#     orderdetails
# JOIN
#     products
# ON
#     orderdetails.ProductID = products.ProductID
# JOIN
#     orders
# ON
#     orderdetails.OrderID = orders.OrderID
# JOIN
#     customers
# ON
#     orders.CustomerID = customers.CustomerID
# GROUP BY
#     customers.CustomerID, customers.CustomerName
# ORDER BY
#     TotalOrderAmount DESC;
# """)


# Average Order Amount
query = text("""
SELECT
    customers.CustomerID,
    customers.CustomerName,
    COUNT(orderdetails.OrderID) AS OrderCount,  -- count of orders
    SUM(products.Price * orderdetails.Quantity) / COUNT(orderdetails.OrderID) AS AveOrderAmount
FROM
    orderdetails
JOIN
    products
ON
    orderdetails.ProductID = products.ProductID
JOIN
    orders
ON
    orderdetails.OrderID = orders.OrderID
JOIN
    customers
ON
    orders.CustomerID = customers.CustomerID
GROUP BY
    customers.CustomerID, customers.CustomerName
ORDER BY
    AveOrderAmount DESC;

""")

# Use a connection from the engine
with engine.connect() as connection:
    df_question5 = pd.read_sql(query, connection)

# Display the results
print(df_question5)

    CustomerID                        CustomerName  OrderCount  AveOrderAmount
0           59                    Piccolo und mehr           4       4014.2500
1           73                       Simons bistro           6       2444.3333
2           62                       Queen Cozinha           9       1991.6667
3           51                      Mère Paillarde          14       1673.8571
4           71                  Save-a-lot Markets          16       1407.2500
..         ...                                 ...         ...             ...
69          29              Galería del gastrónomo           4        147.7500
70          69                    Romero y tomillo           8        116.5000
71          13          Centro comercial Moctezuma           2         63.0000
72          27                      Franchi S.p.A.           1         62.0000
73           2  Ana Trujillo Emparedados y helados           2         55.5000

[74 rows x 4 columns]


|CustomerID                      |CustomerName|AvgOrderAmount|
|--------------------------------|------------|--------------|
|59                              |Piccolo und mehr|4014.2500     |
|73                              |Simons bistro|2444.3333     |
|62                              |Queen Cozinha|1991.6667     |
|51                              |Mère Paillarde|1673.8571     |
|71                              |Save-a-lot Markets|1407.2500     |
|76                              |Suprêmes délices|1345.8333     |
|81                              |Tradição Hipermercados|1315.6667     |
|7                               |Blondel père et fils|1174.4615     |
|89                              |White Clover Markets|1112.5000     |
|55                              |Old World Delicatessen|1079.5000     |
|20                              |Ernst Handel|1018.0000     |
|19                              |Eastern Connection|1004.8000     |
|68                              |Richter Supermarkt|976.6667      |
|72                              |Seven Seas Imports|934.0000      |
|63                              |QUICK-Stop  |908.5500      |
|25                              |Frankenversand|895.0667      |
|75                              |Split Rail Beer & Ale|854.7692      |
|65                              |Rattlesnake Canyon Grocery|838.2727      |
|52                              |Morgenstern Gesundkost|754.0000      |
|9                               |Bon app''   |750.4286      |
|37                              |Hungry Owl All-Night Grocers|733.5714      |
|35                              |HILARIÓN-Abastos|722.8333      |
|33                              |GROSELLA-Restaurante|690.0000      |
|34                              |Hanari Carnes|681.0000      |
|15                              |Comércio Mineiro|677.5000      |
|23                              |Folies gourmandes|672.3333      |
|10                              |Bottom-Dollar Marketse|664.0000      |
|31                              |Gourmet Lanchonetes|641.0000      |
|60                              |Princesa Isabel Vinhoss|628.5000      |
|5                               |Berglunds snabbköp|601.5556      |
|46                              |LILA-Supermercado|564.7692      |
|44                              |Lehmanns Marktstand|545.7500      |
|30                              |Godos Cocina Típica|515.0000      |
|3                               |Antonio Moreno Taquería|504.0000      |
|88                              |Wellington Importadora|503.8333      |
|47                              |LINO-Delicateses|500.0000      |
|56                              |Ottilies Käseladen|500.0000      |
|87                              |Wartian Herkku|492.5833      |
|24                              |Folk och fä HB|480.2222      |
|49                              |Magazzini Alimentari Riuniti|464.2857      |
|86                              |Die Wandernde Kuh|441.9091      |
|41                              |La maison d''Asie|437.8182      |
|8                               |Bólido Comidas preparadas|416.0000      |
|83                              |Vaffeljernet|411.0000      |
|14                              |Chop-suey Chinese|374.8333      |
|80                              |Tortuga Restaurante|357.4167      |


## 6. Sales Analysis with CTEs

Assume we have the Northwind database which contains tables like Orders, OrderDetails, and Products. Create a CTE that calculates the total sales for each product in the year 1997.

In [92]:
import pandas as pd
from sqlalchemy import create_engine, text


query = text("""
SELECT 
    products.ProductID,
    products.ProductName,
    SUM(orderdetails.Quantity) AS TotalQuantity
FROM 
    orders
JOIN 
    orderdetails
ON 
    orders.OrderID = orderdetails.OrderID
JOIN 
    products
ON 
    orderdetails.ProductID = products.ProductID
WHERE 
    YEAR(orders.OrderDate) = 1997
GROUP BY 
    products.ProductID, products.ProductName
ORDER BY 
    TotalQuantity DESC;
    
""")


# Use a connection from the engine
with engine.connect() as connection:
    df_question6 = pd.read_sql(query, connection)

# Display the results
print(df_question6)





    ProductID                       ProductName  TotalQuantity
0          56            Gnocchi di nonna Alice          173.0
1          54                         Tourtière          126.0
2          33                           Geitost          119.0
3           2                             Chang          115.0
4          59              Raclette Courdavault          115.0
5          61                    Sirop d'érable          106.0
6          63                      Vegie-spread          100.0
7          38                     Côte de Blaye           99.0
8          17                      Alice Mutton           97.0
9          35                    Steeleye Stout           95.0
10         21               Sir Rodney's Scones           92.0
11         16                           Pavlova           86.0
12         47                     Zaanse koeken           85.0
13         71                       Fløtemysost           75.0
14         62                    Tarte au sucre        

#### Expected result

|ProductName                     |TotalSales|
|--------------------------------|----------|
|Gnocchi di nonna Alice          |173       |
|Tourtière                       |126       |
|Geitost                         |119       |
|Chang                           |115       |
|Raclette Courdavault            |115       |
|Sirop d'érable                  |106       |
|Vegie-spread                    |100       |
|Côte de Blaye                   |99        |
|Alice Mutton                    |97        |
|Steeleye Stout                  |95        |
|Sir Rodney's Scones             |92        |
|Pavlova                         |86        |
|Zaanse koeken                   |85        |
|Fløtemysost                     |75        |
|Tarte au sucre                  |75        |
